# 天池工业AI大赛-智能制造质量预测

## 数据说明
每条数据包含8029列字段。第一个字段为ID号码，最后一列为要预测的值Y。其余的数据为用于预测Y的变量X。这些变量一共由多道工序组成，字段的名字可以区分不同的工序，例如 210X1, 210X2。300X1,300X2。字段中的TOOL_ID或者Tool为每道工序使用的机台，如果是string类型，需要选手自行进行数字化转换，数据中存在缺失值。初赛总计提供500条数据，复赛数据待定。

- 考虑使用平均值代替缺失值；删除缺失值（下策）
- pca降维
- TOOL_ID数字化转换

In [1]:
import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame

data_train = pd.read_excel("input/训练.xlsx")
data_train.columns

Index(['ID', 'TOOL_ID', '210X1', '210X2', '210X3', '210X4', '210X5', '210X6',
       '210X7', '210X8',
       ...
       '750X1444', '750X1445', '750X1446', '750X1447', '750X1448', '750X1449',
       '750X1450', '750X1451', '750X1452', 'Y'],
      dtype='object', length=8029)

In [2]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Columns: 8029 entries, ID to Y
dtypes: float64(6279), int64(1739), object(11)
memory usage: 30.6+ MB


<font color=red>object(11) : 除去第一列ID后，还有10列与tool相关的字段。</font>

In [3]:
data_train.describe()

210X1       210X2       210X3       210X4       210X5       210X6  \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean    92.272420    0.511932    0.232420    1.626228   57.126160    3.424444   
std     27.471434    0.233265    0.104536    1.090267   18.224157    1.148460   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%    100.100000    0.365750    0.210000    1.019750   56.575000    3.242000   
50%    101.300000    0.506000    0.240000    1.293000   62.130000    3.616000   
75%    101.750000    0.612250    0.270000    1.925750   67.250000    3.864250   
max    102.800000    0.970000    0.410000    3.891000   71.980000    5.173000   

            210X7       210X8       210X9      210X10     ...        750X1444  \
count  500.000000  500.000000  500.000000  500.000000     ...      486.000000   
mean    -0.428580    0.267060   -0.027540   25.854000     ...        0.000694   
std      1.188766    1.092823    0.120268    1.908069     ...        0.000045   
min     -2.450000   -1.920000   -0.220000    0.000000     ...        0.000540   
25%     -1.912500   -0.670000   -0.140000   24.400000     ...        0.000670   
50%     -0.185000    0.400000   -0.015000   26.000000     ...        0.000690   
75%      0.285000    1.220000    0.080000   27.000000     ...        0.000730   
max      2.190000    2.230000    0.170000   29.400000     ...        0.000880   

         750X1445    750X1446    750X1447    750X1448    750X1449    750X1450  \
count  486.000000  486.000000  486.000000  486.000000  486.000000  486.000000   
mean     0.000694   26.010700    0.000694    0.000694   26.010700    0.000694   
std      0.000045    0.349445    0.000045    0.000045    0.349445    0.000045   
min      0.000540   24.900000    0.000540    0.000540   24.900000    0.000540   
25%      0.000670   25.800000    0.000670    0.000670   25.800000    0.000670   
50%      0.000690   26.000000    0.000690    0.000690   26.000000    0.000690   
75%      0.000730   26.300000    0.000730    0.000730   26.300000    0.000730   
max      0.000880   27.100000    0.000880    0.000880   27.100000    0.000880   

         750X1451      750X1452           Y  
count  486.000000  4.860000e+02  500.000000  
mean     0.000694  2.400000e+12    2.846187  
std      0.000045  0.000000e+00    0.200970  
min      0.000540  2.400000e+12    2.326846  
25%      0.000670  2.400000e+12    2.705686  
50%      0.000690  2.400000e+12    2.840442  
75%      0.000730  2.400000e+12    2.980399  
max      0.000880  2.400000e+12    3.454556  

[8 rows x 8018 columns]

In [4]:
data_train.head()

ID TOOL_ID   210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  \
0  ID001       N  102.05  0.465   0.27  1.430  67.45  4.620  -0.54  -1.05   
1  ID002       M  100.95  0.805   0.22  3.477  62.08  3.412  -2.12   1.02   
2  ID003       L   98.56  0.555   0.24  1.172  56.70  3.080  -2.25   0.88   
3  ID004       M  100.35  0.901   0.22  3.631  62.25  3.949  -1.98   0.82   
4  ID005       M  100.25  0.854   0.23  3.429  61.42  3.630  -1.89   1.02   

     ...     750X1444  750X1445  750X1446  750X1447  750X1448  750X1449  \
0    ...      0.00072   0.00072      25.7   0.00072   0.00072      25.7   
1    ...      0.00072   0.00072      25.5   0.00072   0.00072      25.5   
2    ...      0.00064   0.00064      25.2   0.00064   0.00064      25.2   
3    ...      0.00072   0.00072      26.4   0.00072   0.00072      26.4   
4    ...      0.00072   0.00072      26.4   0.00072   0.00072      26.4   

   750X1450  750X1451      750X1452         Y  
0   0.00072   0.00072  2.400000e+12  2.945079  
1   0.00072   0.00072  2.400000e+12  2.955092  
2   0.00064   0.00064  2.400000e+12  2.741264  
3   0.00072   0.00072  2.400000e+12  2.799336  
4   0.00072   0.00072  2.400000e+12  2.692093  

[5 rows x 8029 columns]

In [5]:
data_train.tail()

ID TOOL_ID   210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  \
495  ID711       M  101.85  0.825   0.21  3.439  63.65  3.474   1.07   1.94   
496  ID712       O  100.20  0.612   0.13  1.471  58.17  2.978  -0.82   1.16   
497  ID713       O  100.10  0.622   0.13  1.397  58.09  2.986   0.41   0.41   
498  ID714       J  100.90  0.327   0.41  0.533  71.02  3.592   0.00  -0.29   
499  ID715       O  100.30  0.611   0.13  1.465  58.22  3.029  -0.84   1.02   

       ...     750X1444  750X1445  750X1446  750X1447  750X1448  750X1449  \
495    ...      0.00067   0.00067      26.3   0.00067   0.00067      26.3   
496    ...      0.00081   0.00081      25.9   0.00081   0.00081      25.9   
497    ...      0.00083   0.00083      25.7   0.00083   0.00083      25.7   
498    ...      0.00072   0.00072      26.3   0.00072   0.00072      26.3   
499    ...      0.00074   0.00074      25.8   0.00074   0.00074      25.8   

     750X1450  750X1451      750X1452         Y  
495   0.00067   0.00067  2.400000e+12  2.885743  
496   0.00081   0.00081  2.400000e+12  2.623342  
497   0.00083   0.00083  2.400000e+12  3.015137  
498   0.00072   0.00072  2.400000e+12  2.753286  
499   0.00074   0.00074  2.400000e+12  2.926033  

[5 rows x 8029 columns]

In [6]:
data_train.iloc[301:305,:]

ID TOOL_ID   210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  \
301  ID465       M  101.75  0.819   0.22  3.417  63.44  3.569   1.95   1.80   
302  ID466       L  100.15  0.486   0.24  1.033  57.13  3.527   1.05   1.42   
303  ID466       M    0.00  0.000   0.00  0.000   0.00  0.000   0.85   1.60   
304  ID467       M  101.90  0.811   0.21  3.452  63.58  3.627   1.71   1.88   

       ...     750X1444  750X1445  750X1446  750X1447  750X1448  750X1449  \
301    ...      0.00067   0.00067      26.1   0.00067   0.00067      26.1   
302    ...      0.00074   0.00074      26.2   0.00074   0.00074      26.2   
303    ...      0.00074   0.00074      26.2   0.00074   0.00074      26.2   
304    ...      0.00067   0.00067      26.1   0.00067   0.00067      26.1   

     750X1450  750X1451      750X1452         Y  
301   0.00067   0.00067  2.400000e+12  2.928911  
302   0.00074   0.00074  2.400000e+12  2.878938  
303   0.00074   0.00074  2.400000e+12  2.878938  
304   0.00067   0.00067  2.400000e+12  2.861295  

[4 rows x 8029 columns]

**<font color=red>技术圈问题:</font>** 302与303列ID相同。

**<font color=red>官方解答：</font>** 因为实际生产中这个产品对应的ID出现了2次，L,和M代表不同的字段值，所以这个ID的产品由于某种特殊情况出现了两次。是个别的生产中存在的问题，选手可以结合数据自行判断是否用于建模。

In [7]:
data = data_train.drop(['ID','Y'],axis=1) # 删除第一列ID

In [8]:
data.head()

TOOL_ID   210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  210X9  \
0       N  102.05  0.465   0.27  1.430  67.45  4.620  -0.54  -1.05  -0.13   
1       M  100.95  0.805   0.22  3.477  62.08  3.412  -2.12   1.02   0.08   
2       L   98.56  0.555   0.24  1.172  56.70  3.080  -2.25   0.88   0.17   
3       M  100.35  0.901   0.22  3.631  62.25  3.949  -1.98   0.82   0.08   
4       M  100.25  0.854   0.23  3.429  61.42  3.630  -1.89   1.02   0.08   

       ...       750X1443  750X1444  750X1445  750X1446  750X1447  750X1448  \
0      ...        0.00072   0.00072   0.00072      25.7   0.00072   0.00072   
1      ...        0.00072   0.00072   0.00072      25.5   0.00072   0.00072   
2      ...        0.00064   0.00064   0.00064      25.2   0.00064   0.00064   
3      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   
4      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   

   750X1449  750X1450  750X1451      750X1452  
0      25.7   0.00072   0.00072  2.400000e+12  
1      25.5   0.00072   0.00072  2.400000e+12  
2      25.2   0.00064   0.00064  2.400000e+12  
3      26.4   0.00072   0.00072  2.400000e+12  
4      26.4   0.00072   0.00072  2.400000e+12  

[5 rows x 8027 columns]

###  <font color=red>每道工序使用的机台字段：</font> 
共10项：'TOOL_ID','Tool','Tool (#2)','TOOL_ID (#1)','TOOL_ID (#2)','TOOL_ID (#3)','tool (#1)','TOOL','TOOL (#1)','TOOL (#2)'

In [9]:
data = data_train.drop(['ID','Y','TOOL_ID','Tool','Tool (#2)','TOOL_ID (#1)','TOOL_ID (#2)','TOOL_ID (#3)','tool (#1)','TOOL','TOOL (#1)','TOOL (#2)'],axis=1) # 删除第一列ID
data.head()

210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  210X9  210X10  \
0  102.05  0.465   0.27  1.430  67.45  4.620  -0.54  -1.05  -0.13    26.3   
1  100.95  0.805   0.22  3.477  62.08  3.412  -2.12   1.02   0.08    28.2   
2   98.56  0.555   0.24  1.172  56.70  3.080  -2.25   0.88   0.17    26.6   
3  100.35  0.901   0.22  3.631  62.25  3.949  -1.98   0.82   0.08    25.2   
4  100.25  0.854   0.23  3.429  61.42  3.630  -1.89   1.02   0.08    27.3   

       ...       750X1443  750X1444  750X1445  750X1446  750X1447  750X1448  \
0      ...        0.00072   0.00072   0.00072      25.7   0.00072   0.00072   
1      ...        0.00072   0.00072   0.00072      25.5   0.00072   0.00072   
2      ...        0.00064   0.00064   0.00064      25.2   0.00064   0.00064   
3      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   
4      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   

   750X1449  750X1450  750X1451      750X1452  
0      25.7   0.00072   0.00072  2.400000e+12  
1      25.5   0.00072   0.00072  2.400000e+12  
2      25.2   0.00064   0.00064  2.400000e+12  
3      26.4   0.00072   0.00072  2.400000e+12  
4      26.4   0.00072   0.00072  2.400000e+12  

[5 rows x 8017 columns]

In [10]:
data.head()

210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  210X9  210X10  \
0  102.05  0.465   0.27  1.430  67.45  4.620  -0.54  -1.05  -0.13    26.3   
1  100.95  0.805   0.22  3.477  62.08  3.412  -2.12   1.02   0.08    28.2   
2   98.56  0.555   0.24  1.172  56.70  3.080  -2.25   0.88   0.17    26.6   
3  100.35  0.901   0.22  3.631  62.25  3.949  -1.98   0.82   0.08    25.2   
4  100.25  0.854   0.23  3.429  61.42  3.630  -1.89   1.02   0.08    27.3   

       ...       750X1443  750X1444  750X1445  750X1446  750X1447  750X1448  \
0      ...        0.00072   0.00072   0.00072      25.7   0.00072   0.00072   
1      ...        0.00072   0.00072   0.00072      25.5   0.00072   0.00072   
2      ...        0.00064   0.00064   0.00064      25.2   0.00064   0.00064   
3      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   
4      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   

   750X1449  750X1450  750X1451      750X1452  
0      25.7   0.00072   0.00072  2.400000e+12  
1      25.5   0.00072   0.00072  2.400000e+12  
2      25.2   0.00064   0.00064  2.400000e+12  
3      26.4   0.00072   0.00072  2.400000e+12  
4      26.4   0.00072   0.00072  2.400000e+12  

[5 rows x 8017 columns]

In [11]:
data.describe()

210X1       210X2       210X3       210X4       210X5       210X6  \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean    92.272420    0.511932    0.232420    1.626228   57.126160    3.424444   
std     27.471434    0.233265    0.104536    1.090267   18.224157    1.148460   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%    100.100000    0.365750    0.210000    1.019750   56.575000    3.242000   
50%    101.300000    0.506000    0.240000    1.293000   62.130000    3.616000   
75%    101.750000    0.612250    0.270000    1.925750   67.250000    3.864250   
max    102.800000    0.970000    0.410000    3.891000   71.980000    5.173000   

            210X7       210X8       210X9      210X10      ...       \
count  500.000000  500.000000  500.000000  500.000000      ...        
mean    -0.428580    0.267060   -0.027540   25.854000      ...        
std      1.188766    1.092823    0.120268    1.908069      ...        
min     -2.450000   -1.920000   -0.220000    0.000000      ...        
25%     -1.912500   -0.670000   -0.140000   24.400000      ...        
50%     -0.185000    0.400000   -0.015000   26.000000      ...        
75%      0.285000    1.220000    0.080000   27.000000      ...        
max      2.190000    2.230000    0.170000   29.400000      ...        

         750X1443    750X1444    750X1445    750X1446    750X1447    750X1448  \
count  486.000000  486.000000  486.000000  486.000000  486.000000  486.000000   
mean     0.000694    0.000694    0.000694   26.010700    0.000694    0.000694   
std      0.000045    0.000045    0.000045    0.349445    0.000045    0.000045   
min      0.000540    0.000540    0.000540   24.900000    0.000540    0.000540   
25%      0.000670    0.000670    0.000670   25.800000    0.000670    0.000670   
50%      0.000690    0.000690    0.000690   26.000000    0.000690    0.000690   
75%      0.000730    0.000730    0.000730   26.300000    0.000730    0.000730   
max      0.000880    0.000880    0.000880   27.100000    0.000880    0.000880   

         750X1449    750X1450    750X1451      750X1452  
count  486.000000  486.000000  486.000000  4.860000e+02  
mean    26.010700    0.000694    0.000694  2.400000e+12  
std      0.349445    0.000045    0.000045  0.000000e+00  
min     24.900000    0.000540    0.000540  2.400000e+12  
25%     25.800000    0.000670    0.000670  2.400000e+12  
50%     26.000000    0.000690    0.000690  2.400000e+12  
75%     26.300000    0.000730    0.000730  2.400000e+12  
max     27.100000    0.000880    0.000880  2.400000e+12  

[8 rows x 8017 columns]

In [12]:
from sklearn.preprocessing import Imputer    
import numpy as np  
 
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
#上面'NAN'表示无效值在数据文件中的标识是'NAN',strategy='mean'表示用全局平均值代替无效值，axis=0表示对列进行处理  
  
imp.fit(data)  
#训练一个缺失值处理模型  


Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [13]:
print(imp.transform(data)) 

[[  1.02050000e+02   4.65000000e-01   2.70000000e-01 ...,   7.20000000e-04
    7.20000000e-04   2.40000000e+12]
 [  1.00950000e+02   8.05000000e-01   2.20000000e-01 ...,   7.20000000e-04
    7.20000000e-04   2.40000000e+12]
 [  9.85600000e+01   5.55000000e-01   2.40000000e-01 ...,   6.40000000e-04
    6.40000000e-04   2.40000000e+12]
 ..., 
 [  1.00100000e+02   6.22000000e-01   1.30000000e-01 ...,   8.30000000e-04
    8.30000000e-04   2.40000000e+12]
 [  1.00900000e+02   3.27000000e-01   4.10000000e-01 ...,   7.20000000e-04
    7.20000000e-04   2.40000000e+12]
 [  1.00300000e+02   6.11000000e-01   1.30000000e-01 ...,   7.40000000e-04
    7.40000000e-04   2.40000000e+12]]


In [14]:
outfile = imp.transform(data)  
  
#存储到本地  
np.savetxt('outfile.csv', outfile, delimiter=',')  

In [15]:
data_x = pd.read_csv("outfile.csv",header=None)

In [16]:
data_x.describe() #describe()

0           1           2           3           4           5     \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean    92.272420    0.511932    0.232420    1.626228   57.126160    3.424444   
std     27.471434    0.233265    0.104536    1.090267   18.224157    1.148460   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%    100.100000    0.365750    0.210000    1.019750   56.575000    3.242000   
50%    101.300000    0.506000    0.240000    1.293000   62.130000    3.616000   
75%    101.750000    0.612250    0.270000    1.925750   67.250000    3.864250   
max    102.800000    0.970000    0.410000    3.891000   71.980000    5.173000   

             6           7           8           9         ...       \
count  500.000000  500.000000  500.000000  500.000000      ...        
mean    -0.428580    0.267060   -0.027540   25.854000      ...        
std      1.188766    1.092823    0.120268    1.908069      ...        
min     -2.450000   -1.920000   -0.220000    0.000000      ...        
25%     -1.912500   -0.670000   -0.140000   24.400000      ...        
50%     -0.185000    0.400000   -0.015000   26.000000      ...        
75%      0.285000    1.220000    0.080000   27.000000      ...        
max      2.190000    2.230000    0.170000   29.400000      ...        

             7945        7946        7947        7948        7949        7950  \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean     0.000694    0.000694    0.000694   26.010700    0.000694    0.000694   
std      0.000044    0.000044    0.000044    0.344508    0.000044    0.000044   
min      0.000540    0.000540    0.000540   24.900000    0.000540    0.000540   
25%      0.000670    0.000670    0.000670   25.800000    0.000670    0.000670   
50%      0.000694    0.000694    0.000694   26.010700    0.000694    0.000694   
75%      0.000730    0.000730    0.000730   26.300000    0.000730    0.000730   
max      0.000880    0.000880    0.000880   27.100000    0.000880    0.000880   

             7951        7952        7953          7954  
count  500.000000  500.000000  500.000000  5.000000e+02  
mean    26.010700    0.000694    0.000694  2.400000e+12  
std      0.344508    0.000044    0.000044  0.000000e+00  
min     24.900000    0.000540    0.000540  2.400000e+12  
25%     25.800000    0.000670    0.000670  2.400000e+12  
50%     26.010700    0.000694    0.000694  2.400000e+12  
75%     26.300000    0.000730    0.000730  2.400000e+12  
max     27.100000    0.000880    0.000880  2.400000e+12  

[8 rows x 7955 columns]

In [23]:
data.head()

210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  210X9  210X10  \
0  102.05  0.465   0.27  1.430  67.45  4.620  -0.54  -1.05  -0.13    26.3   
1  100.95  0.805   0.22  3.477  62.08  3.412  -2.12   1.02   0.08    28.2   
2   98.56  0.555   0.24  1.172  56.70  3.080  -2.25   0.88   0.17    26.6   
3  100.35  0.901   0.22  3.631  62.25  3.949  -1.98   0.82   0.08    25.2   
4  100.25  0.854   0.23  3.429  61.42  3.630  -1.89   1.02   0.08    27.3   

       ...       750X1443  750X1444  750X1445  750X1446  750X1447  750X1448  \
0      ...        0.00072   0.00072   0.00072      25.7   0.00072   0.00072   
1      ...        0.00072   0.00072   0.00072      25.5   0.00072   0.00072   
2      ...        0.00064   0.00064   0.00064      25.2   0.00064   0.00064   
3      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   
4      ...        0.00072   0.00072   0.00072      26.4   0.00072   0.00072   

   750X1449  750X1450  750X1451      750X1452  
0      25.7   0.00072   0.00072  2.400000e+12  
1      25.5   0.00072   0.00072  2.400000e+12  
2      25.2   0.00064   0.00064  2.400000e+12  
3      26.4   0.00072   0.00072  2.400000e+12  
4      26.4   0.00072   0.00072  2.400000e+12  

[5 rows x 8017 columns]

In [24]:
data.T

0             1             2             3    \
210X1     1.020500e+02  1.009500e+02  9.856000e+01  1.003500e+02   
210X2     4.650000e-01  8.050000e-01  5.550000e-01  9.010000e-01   
210X3     2.700000e-01  2.200000e-01  2.400000e-01  2.200000e-01   
210X4     1.430000e+00  3.477000e+00  1.172000e+00  3.631000e+00   
210X5     6.745000e+01  6.208000e+01  5.670000e+01  6.225000e+01   
210X6     4.620000e+00  3.412000e+00  3.080000e+00  3.949000e+00   
210X7    -5.400000e-01 -2.120000e+00 -2.250000e+00 -1.980000e+00   
210X8    -1.050000e+00  1.020000e+00  8.800000e-01  8.200000e-01   
210X9    -1.300000e-01  8.000000e-02  1.700000e-01  8.000000e-02   
210X10    2.630000e+01  2.820000e+01  2.660000e+01  2.520000e+01   
210X11    2.795000e+01  2.427000e+01  2.451000e+01  2.438000e+01   
210X12    5.320000e-01  2.653000e+00  5.230000e-01  2.582000e+00   
210X13    7.700000e-02  7.200000e-02  7.600000e-02  7.400000e-02   
210X14    7.900000e-02  6.500000e-02  7.200000e-02  6.800000e-02   
210X15    7.800000e-02  7.300000e-02  7.700000e-02  7.400000e-02   
210X16    7.900000e-02  6.700000e-02  7.300000e-02  6.700000e-02   
210X17    7.500000e+02  7.500000e+02  7.500000e+02  7.500000e+02   
210X18    4.000000e-01  4.000000e-01  3.980000e-01  4.000000e-01   
210X19    2.000000e+00  2.000000e+00  2.000000e+00  2.000000e+00   
210X20    1.430000e+01  1.390000e+01  1.440000e+01  1.400000e+01   
210X21    1.400000e+01  1.400000e+01  1.400000e+01  1.400000e+01   
210X22    1.400000e+01  1.400000e+01  1.400000e+01  1.400000e+01   
210X23    1.420000e+01  1.390000e+01  1.470000e+01  1.380000e+01   
210X24    2.017073e+13  2.017073e+13  2.017073e+13  2.017072e+13   
210X25    6.810000e-01  7.240000e-01  7.540000e-01  7.200000e-01   
210X26    6.750000e-01  7.070000e-01  7.700000e-01  7.040000e-01   
210X27    4.000000e-03  4.000000e-03  3.000000e-03  4.000000e-03   
210X28    8.000000e-03  3.000000e-03  4.000000e-03  4.000000e-03   
210X29    1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03   
210X30    1.000000e-03  1.000000e-03  1.000000e-03  1.000000e-03   
...                ...           ...           ...           ...   
750X1423  3.800000e-05  3.700000e-05  9.700000e-04  1.100000e-04   
750X1424  7.200000e-04  7.200000e-04  6.400000e-04  7.200000e-04   
750X1425  3.800000e-05  3.700000e-05  9.700000e-04  1.100000e-04   
750X1426  3.800000e-05  3.700000e-05  9.700000e-04  1.100000e-04   
750X1427  7.200000e-04  7.200000e-04  6.400000e-04  7.200000e-04   
750X1428  3.800000e-05  3.700000e-05  9.700000e-04  1.100000e-04   
750X1429  3.800000e-05  3.700000e-05  9.700000e-04  1.100000e-04   
750X1430  2.107000e+02  2.125000e+02  2.151000e+02  2.042000e+02   
750X1431  2.107000e+02  2.125000e+02  2.151000e+02  2.042000e+02   
750X1432  1.400000e+00  1.400000e+00  1.400000e+00  1.400000e+00   
750X1433  1.400000e+00  1.400000e+00  1.400000e+00  1.400000e+00   
750X1434  1.400000e+00  1.400000e+00  1.400000e+00  1.400000e+00   
750X1435  3.800000e-05  3.700000e-05  9.700000e-04  1.100000e-04   
750X1436  1.400000e+00  1.400000e+00  1.400000e+00  1.400000e+00   
750X1437  1.400000e+00  1.400000e+00  1.400000e+00  1.400000e+00   
750X1438  3.800000e-05  3.700000e-05  9.700000e-04  1.100000e-04   
750X1439  1.400000e+00  1.400000e+00  1.400000e+00  1.400000e+00   
750X1440  1.400000e+00  1.400000e+00  1.400000e+00  1.400000e+00   
750X1441  2.107000e+02  2.125000e+02  2.151000e+02  2.042000e+02   
750X1442  2.400000e+12  2.400000e+12  2.400000e+12  2.400000e+12   
750X1443  7.200000e-04  7.200000e-04  6.400000e-04  7.200000e-04   
750X1444  7.200000e-04  7.200000e-04  6.400000e-04  7.200000e-04   
750X1445  7.200000e-04  7.200000e-04  6.400000e-04  7.200000e-04   
750X1446  2.570000e+01  2.550000e+01  2.520000e+01  2.640000e+01   
750X1447  7.200000e-04  7.200000e-04  6.400000e-04  7.200000e-04   
750X1448  7.200000e-04  7.200000e-04  6.400000e-04  7.200000e-04   
750X1449  2.570000e+01  2.550000e+01  2.520000e+01  2.640000e+01   
750X145

In [27]:
IsDuplicated=data.T.duplicated() 
print(IsDuplicated)

210X1       False
210X2       False
210X3       False
210X4       False
210X5       False
210X6       False
210X7       False
210X8       False
210X9       False
210X10      False
210X11      False
210X12      False
210X13      False
210X14      False
210X15      False
210X16      False
210X17      False
210X18      False
210X19      False
210X20      False
210X21      False
210X22       True
210X23      False
210X24      False
210X25      False
210X26      False
210X27      False
210X28      False
210X29      False
210X30      False
            ...  
750X1423     True
750X1424     True
750X1425     True
750X1426     True
750X1427     True
750X1428     True
750X1429     True
750X1430     True
750X1431     True
750X1432     True
750X1433     True
750X1434     True
750X1435     True
750X1436     True
750X1437     True
750X1438     True
750X1439     True
750X1440     True
750X1441     True
750X1442     True
750X1443     True
750X1444     True
750X1445     True
750X1446     True
750X1447  

In [28]:
data_drop = data.T.drop_duplicates()

In [29]:
data_drop.T

210X1  210X2  210X3  210X4  210X5  210X6  210X7  210X8  210X9  210X10  \
0    102.05  0.465   0.27  1.430  67.45  4.620  -0.54  -1.05  -0.13    26.3   
1    100.95  0.805   0.22  3.477  62.08  3.412  -2.12   1.02   0.08    28.2   
2     98.56  0.555   0.24  1.172  56.70  3.080  -2.25   0.88   0.17    26.6   
3    100.35  0.901   0.22  3.631  62.25  3.949  -1.98   0.82   0.08    25.2   
4    100.25  0.854   0.23  3.429  61.42  3.630  -1.89   1.02   0.08    27.3   
5    100.55  0.882   0.22  3.462  61.85  3.747  -2.10   0.93   0.08    25.3   
6    102.20  0.440   0.27  1.429  67.38  4.628  -0.20  -0.44  -0.13    26.5   
7    102.25  0.420   0.27  1.312  67.47  4.614  -0.99  -0.99  -0.13    27.1   
8    102.25  0.421   0.27  1.311  67.44  4.631   0.32  -1.19  -0.14    25.9   
9    100.85  0.958   0.22  3.705  62.88  4.184  -2.14   0.99   0.08    27.7   
10    98.95  0.525   0.25  1.132  57.14  2.992  -2.04   0.30   0.15    27.3   
11   101.50  0.312   0.40  0.554  71.63  3.697   0.20  -0.44  -0.16    24.3   
12   101.45  0.311   0.40  0.560  71.61  3.712   0.00  -0.61  -0.18    24.9   
13   101.45  0.314   0.40  0.571  71.67  3.715   0.03  -0.58  -0.16    25.0   
14   101.75  0.808   0.22  3.487  62.54  3.524  -1.96   1.08   0.08    26.4   
15    99.10  0.569   0.24  1.134  56.66  3.460  -1.96   0.70   0.14    26.0   
16   102.10  0.439   0.27  1.391  67.24  4.618  -0.08  -0.96  -0.13    26.3   
17   101.95  0.843   0.22  3.704  63.19  3.745  -2.06   0.93   0.08    27.7   
18    99.13  0.575   0.24  1.224  56.64  3.431  -1.98   0.62   0.14    27.3   
19   102.15  0.448   0.27  1.411  67.26  4.627  -0.68  -0.61  -0.10    26.2   
20   101.50  0.794   0.22  3.396  62.02  3.372  -1.96   1.19   0.08    28.0   
21    99.11  0.571   0.24  1.208  56.61  3.431  -1.96   0.65   0.14    28.4   
22   102.15  0.461   0.27  1.516  67.38  4.603  -0.68  -1.05  -0.12    26.2   
23   101.90  0.843   0.22  3.689  63.01  3.763  -2.02   0.99   0.07    27.7   
24   102.35  0.403   0.27  1.286  67.46  4.585  -0.85  -0.90  -0.13    26.5   
25    99.30  0.508   0.25  1.139  57.23  3.132  -2.10   0.53   0.15    27.0   
26   102.05  0.459   0.27  1.470  67.29  4.633  -0.06  -0.84  -0.13    26.3   
27    99.58  0.514   0.24  1.210  57.35  3.170  -1.98   0.97   0.14    28.1   
28   102.35  0.402   0.27  1.121  67.36  4.582  -0.61  -1.05  -0.13    27.1   
29   102.35  0.389   0.27  1.086  67.36  4.561   0.31  -1.08  -0.13    26.3   
..      ...    ...    ...    ...    ...    ...    ...    ...    ...     ...   
470  100.20  0.620   0.23  1.486  55.05  3.914   0.45   1.57   0.07    27.3   
471  100.50  0.822   0.22  2.998  61.72  3.151   1.26   2.20  -0.03    25.7   
472  100.85  0.525   0.24  1.359  55.80  3.950  -2.21   0.39   0.14    29.4   
473  100.95  0.541   0.13  1.337  58.20  2.970  -0.92   1.66   0.08    24.3   
474   85.53  0.546   0.12  2.062  24.13  2.549   0.53   1.16   0.02    23.4   
475  102.20  0.468   0.25  1.458  61.34  2.952  -0.35  -0.58  -0.13    27.6   
476  100.90  0.530   0.13  1.232  58.23  2.978  -0.75   1.22   0.06    24.6   
477  101.25  0.526   0.23  1.520  54.63  3.795   0.89   1.71   0.04    26.4   
478  100.35  0.616   0.13  1.452  57.70  2.859   0.12   1.05   0.06    23.8   
479  100.55  0.566   0.13  1.237  58.13  2.886   0.23   1.69   0.07    23.4   
480  100.65  0.522   0.13  0.997  57.84  2.942  -0.77   1.40   0.08    24.5   
481  100.65  0.540   0.13  0.977  57.80  2.915  -0.82   1.25   0.06    26.3   
482  100.05  0.615   0.23  1.436  55.09  3.961   0.17   1.39   0.05    26.8   
483  100.10  0.585   0.24  1.281  55.96  3.694  -2.39   0.10   0.16    27.3   
484  102.20  0.442   0.26  1.432  61.03  3.165  -0.95  -0.90  -0.13    26.7   
485  100.20  0.576   0.14  1.230  57.81  2.796   0.20   1.25   0.06    22.9   
486  102.25  0.872   0.21  3.568  62.55  3.480   1.69   1.97  -0.03    25.4   
487  100.95  0.508   0.13  0.950  57.72  2.963  -0.85   1.69   0.09    24.3   
488    0.00  0.000   0.00  0.000   0.00  0.000   1.03  -

In [22]:
from sklearn.decomposition import PCA
pca = PCA(n_components='mle')
pca.fit(data_x)
print(pca.explained_variance_ratio_)
#print pca.explained_variance_ratio_
#print pca.explained_variance_

TypeError: '>=' not supported between instances of 'str' and 'int'

In [ ]:
# 一个处理缺失值的例子
import numpy as np
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit([[1, 2], [np.nan, 3], [7, 6]])
Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))                           